## Notes for FINAL PROJECT

<i>Notes for the Final Project--experiments, broken code, etcetera.
</i>

In [1]:
#import Panda and Numpy and other libraries for use
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 
#to view our charts inline, not a separate window

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn import tree 
#for our secondary regression test, seeing if 
#tree or LogReg give vastly different results

In [2]:
#defining data location
location = "Kickstartercombo.csv"
ks_df = pd.read_csv(location, low_memory=False)
ks_df.head()

FileNotFoundError: File b'Kickstartercombo.csv' does not exist

In [ ]:
ks_df.shape

In [ ]:
ks_df.columns

#### Columns that seem to be important, just from looking over this.
<p>Goal</p>
Pledged
<p>State (failed, etc)</p>
pledged_real
<p>goal</p>
Backers
<p>(??)Launched</p>

#### Columns we can probably ignore
ID,
NAME,
Category,
Country,
State,
Launched,

In [ ]:
ks_df.isnull().sum()

In [ ]:
#seeing what the average number of backers were for projects by status
#I suspect that determines how successful a project is,
#and whether we would want to invest
ks_df['backers'].groupby(ks_df['state']).mean()


In [ ]:
#money asked for vs. money raised
def cash(fn,ln):
    return fn / ln
 
ks_df['money'] = cash(ks_df['pledged'],ks_df['goal'])
ks_df.head()

In [ ]:
#saving that averages list as a spreasdheet for demo in Tableau later
#but I cannot get it to work, so...well
#backing = ks_df['backers'].groupby(ks_df['state']).mean()

#back = pd.DataFrame(data = backing, columns=['State','Avg. Backing Count']

#writer = pd.ExcelWriter('avg-back.xlsx', engine='xlsxwriter')
#back.to_excel(writer, sheet_name='Sheet1')
#writer.save()

In [ ]:
#creating a dataframe with the more relevant data for backers
backers = pd.DataFrame(data=ks_df, columns=['money','backers','state','goal','pledged'])
backers.head()

In [ ]:
#dummies for state
fail = pd.get_dummies(data=backers,columns=['state'])
fail.head()

In [ ]:
#how does backers influence success just at a glance? Does it...
plt.figure(figsize=(17,9))

ax = sns.kdeplot(fail['backers'][fail['state_successful'] == 1], #successful
                 color="darkturquoise",
                 shade=True)

sns.kdeplot(fail['backers'][fail['state_successful'] == 0], #pretty much losers
            color="lightcoral",
            shade=True)

plt.legend (['Successful', 'Failed'])
plt.title("Density Plot of # of Backers on State")

ax.set(xlabel='Backer Count')
#plt.show()

In [ ]:
fail.isnull().sum()

In [ ]:
fail2 = fail.drop(['state_canceled','state_failed','state_live','state_suspended','state_undefined'], axis=1)
fail2.head()

In [ ]:
#starting logistic regression
y = fail2['state_successful']

In [ ]:
#independent variable
X = fail2.drop(['state_successful'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=32342)



In [ ]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

In [ ]:
LogReg.score(X_train, y_train)

In [ ]:
y_pred = LogReg.predict(X_test)

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Failure','Predicted Success'],
    index=['True Failure','True Success']
)
cm

In [ ]:
LogReg.score(X_test, y_test)

In [ ]:
print(classification_report(y_test, y_pred))

### Hallelujah we got it to work (I think), and it even got a 98% accuracy!

In [ ]:
fail2.columns

In [ ]:
#Now to test whether or not money raised has an impact!
#maybe I just did this so that it made sense and worked to compensate
#for the epic failure of yesterday's Project 3...
X2 = fail2.drop(['money'], axis=1)


In [ ]:
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=0.4, random_state=32342)



In [ ]:
LogReg.fit(X2_train, y_train)

In [ ]:
LogReg.score(X2_train, y_train)

In [ ]:
y2_pred = LogReg.predict(X2_test)

In [ ]:
cm2 = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Failure','Predicted Success'],
    index=['True Failure','True Success']
)
cm2

In [ ]:
LogReg.score(X2_test,y_test)

In [ ]:
print(classification_report(y_test,y2_pred))

In [ ]:
#more money more problems?
X3 = fail2.drop(['state_successful','goal','money'], axis=1)


In [ ]:
y3 = fail2['state_successful']

In [ ]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X, y, test_size=0.4, random_state=32342)
#now it is seeing if asking for too much money is recipe for disaster

In [ ]:
LogReg.fit(X3_train, y3_train)

In [ ]:
LogReg.score(X3_train,y3_train)

In [ ]:
y3_pred = LogReg.predict(X3_test)

In [ ]:
cm3 = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Not Survival', 'Predicted Survival'],
    index=['True Not Survival', 'True Survival']
)

cm3

In [ ]:
LogReg.score(X3_test, y3_test)

In [ ]:
print(classification_report(y3_test, y3_pred))

## Field (of Project)

In [ ]:
ks_df.columns

In [ ]:
field = pd.DataFrame(data=ks_df, columns=['main_category','category','money','state'])
field.head()

In [ ]:
categ = pd.get_dummies(data= field, columns=['state'])
categ.head()

In [ ]:
categ = categ.drop(['main_category','state_canceled','state_failed','state_live','state_suspended','state_undefined'], axis=1)
categ.head()

In [ ]:
categ = pd.get_dummies(data=categ, columns=['category'])
categ.head()

In [ ]:
y4 = categ['state_successful']

In [ ]:
X4 = categ.drop(['state_successful'],axis=1)

In [ ]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size = 0.4, random_state=33423)


In [ ]:
LogReg.fit(X4_train, y4_train)

In [ ]:
LogReg.score(X4_train, y4_train)

In [ ]:
y4_pred = LogReg.predict(X4_test)

In [ ]:
cm4 = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Not Survival', 'Predicted Survival'],
    index=['True Not Survival', 'True Survival']
)

cm4

In [ ]:
LogReg.score(X4_test,y4_test)

In [ ]:
print(classification_report(y4_test, y4_pred))

## Less accurate than other measures, but still useful!

In [ ]:
#saving that averages list as a spreasdheet for demo in Tableau later
#but I cannot get it to work, so...well
#backing = ks_df['fail'].groupby(ks_df['state']).mean()


#writer = pd.ExcelWriter('failure.xlsx', engine='xlsxwriter')
#fail.to_excel("failure.xlsx", engine='xlsxwriter')

#with pd.ExcelWriter('final.xlsx') as writer:
backers.to_csv('backers.csv',index=False,header=True)
fail2.to_csv('fail2.csv', index=False,header=True)
categ.to_csv('category.csv', index=False,header=True)

In [ ]:
#names = ['Tom','Joe']
#grades = [44,32]
#GradeList = list(zip(names,grades))

#df = pd.DataFrame(data = GradeList, columns =['Names','Grades'])
#writer = pd.csvWriter('grades324.csv', engine='csvwriter')
#df.to_csv(writer)
#writer.save()